https://wikidocs.net/157896

In [ ]:
# pororo를 사용하려면 torch == 1.6.0버전에서 동작가능
# => colab에는 지원하지 않음 => 설치 불가능

In [13]:
import torch
print(torch.__version__)


2.4.1+cu121


In [14]:
!pip uninstall torch -y  # 기존 torch 제거
!pip install torch==1.6.0  # 지정된 버전 설치
!pip install Pororo  # Pororo 설치


Found existing installation: torch 2.4.1+cu121
Uninstalling torch-2.4.1+cu121:
  Successfully uninstalled torch-2.4.1+cu121
ERROR: Could not find a version that satisfies the requirement torch==1.6.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torch==1.6.0
  Using cached pororo-0.4.2-py3-none-any.whl.metadata (9.0 kB)
INFO: pip is looking at multiple versions of pororo to determine which version is compatible with other requirements. This could take a while.
  Using cached pororo-0.4.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached pororo-0.4.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached pororo-0.3.5-py3-none-any.whl.metadata (8.8 kB)
  Using cached pororo-0.3.4-py3-none-any.whl.metadata (8.8 kB)
  Using cached pororo-0.3.3-py3-none-any.whl.metadata (8.8 kB)
  Using cached pororo-0.3.2-py3-none-any.whl.metadata (8.5 kB)
  Using cached pororo

In [9]:
from pororo import Pororo
import pandas as pd
from sentence_transformers import util
import torch
import numpy as np
import urllib.request


ModuleNotFoundError: No module named 'pororo'

In [3]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")

Chatbot_Data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

Pororo 를 사용하여 모든 질문에 대한 텍스트 유사도를 계산하고 별도의 컬럼에 저장

In [ ]:
sTe = Pororo(task="sentence_embedding", lang="ko")
Chatbot_Data['EmbVector'] = Chatbot_Data['Q'].progress_map(lambda x : sTe(x))


In [ ]:
Chatbot_Data.head()


In [ ]:
def chat(sent="0"):
  while 1:
    q = input("user > ").strip()
    if q == "quit":
      break
    a = ""
    # Pororo Sentense Embedding으로 텍스트 유사도를 구합니다.
    q = sTe(q)
    # 질문을 Tensor로 바꿉니다.
    q = torch.tensor(q)
    # 코사인 유사도
    cos_sim = util.pytorch_cos_sim(q, EmbData)

    #유사도가 가장 비슷한 질문 인덱스를 구합니다.
    best_sim_idx = int(np.argmax(cos_sim))

    # 질문의 유사도와 가장 비슷한  답변 제공
    answer = Chatbot_Data['A'][best_sim_idx]
    print(answer)


In [ ]:
# 챗봇을 실행 합니다.
chat()


BERT 실습

BERT와 GPT는 둘 다 사전 학습된 언어 모델이지만, 몇 가지 차이점이 있습니다.

**공통점**

- 사전 학습: 두 모델 모두 대규모 텍스트 데이터셋을 사용하여 미리 학습되어 있으며, 다양한 자연어 처리 작업에 활용될 수 있습니다.
- 전이 학습: 특정 도메인이나 작업에 맞춰 파인튜닝하여 성능을 개선할 수 있습니다. 즉, 사전 학습된 모델을 특정 작업에 맞게 조정할 수 있습니다.

**차이점**

*모델 구조:

- BERT: 양방향(Bidirectional) 트랜스포머 모델로, 문맥을 이해하는 데 강력합니다. 입력 문장의 모든 단어를 동시에 고려하여 문맥을 파악합니다.
- GPT: 단방향(Autoregressive) 트랜스포머 모델로, 다음 단어를 예측하는 데 초점을 맞춥니다. 이전 단어들만을 기반으로 다음 단어를 예측하는 방식입니다.


*적용 작업:

- BERT: 주로 문장 분류, 개체 인식, 질문-답변 등 다양한 다운스트림 작업에 사용됩니다.
- GPT: 텍스트 생성, 대화 모델 등에서 주로 사용되며, 자연스러운 대화 흐름을 생성하는 데 강점이 있습니다.


**결론**

BERT와 GPT 모두 기본적인 이해를 바탕으로 답변을 생성하지만, 특정 도메인이나 작업에 맞춰 파인튜닝할 수 있다는 점에서 비슷합니다. 따라서 원하는 결과를 얻기 위해 각 모델의 특성에 맞춰 조정할 수 있습니다.

In [15]:
!pip install transformers

In [1]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [2]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTo

In [18]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [0]


In [19]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [1]


https://wikidocs.net/154530

In [11]:
!pip install sentence_transformers -q

In [4]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer


In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

 https://huggingface.co/models?library=sentence-transformers

In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
print(len(train_data))

11823


In [10]:
batch_size = 32  # 적절한 배치 크기로 조정
embeddings = []

for i in range(0, len(train_data), batch_size):
    batch = train_data['Q'][i:i + batch_size].tolist()
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

train_data['embedding'] = embeddings


KeyboardInterrupt: 

In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)


In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))


In [ ]:
def return_answer(question)
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']


In [ ]:
return_answer('너는 누구니?')
